In [5]:
import nltk
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Bidirectional, BatchNormalization
import tarfile
import os
from six.moves import urllib
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
import os
import numpy as np

data_dir = './data/aclImdb/'
names = ['neg', 'pos']

def loading_text(path, folders):
    texts, labels = [], []
    for idx, label in enumerate(folders):
        folder_path = os.path.join(path, label)
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r') as file:
                texts.append(file.read())
            labels.append(idx)

    return texts, np.array(labels).astype(np.int64)

x_train, y_train = loading_text(os.path.join(data_dir, 'train'), names)
x_test, y_test = loading_text(os.path.join(data_dir, 'test'), names)



In [147]:
REPLACE_REGEX = re.compile(r"[.;:!\'?,\"()\[\]]|(<br\s*/><br\s*/>)|(\-)|(\/)")
#REPLACE_REGEX = re.compile(r"[;:!\'?\"()\[\]]|(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocessing_text(reviews):
    reviews = [REPLACE_REGEX.sub(" ", line.lower()) for line in reviews]
    return reviews

#processed_train =preprocessing_text(x_train)
processed_test = preprocessing_text(x_test)



In [148]:
nltk.download('stopwords')
english_stop_words = set(stopwords.words('english'))


def remove_stop_words(corpus):
    removed_stop_words = [' '.join([word for word in review.split() if word not in english_stop_words])
                         for review in corpus]
    return removed_stop_words

no_stop_words_test = remove_stop_words(processed_test)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/abdulraqeeb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [149]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')

def lemmatized_operation(corpus):
    lemmatizer = WordNetLemmatizer()
    lemmatized_reviews = [' '.join(lemmatizer.lemmatize(word) for word in nltk.word_tokenize(review)) for review in corpus]
    return lemmatized_reviews

lemmatized_reviews = lemmatized_operation(no_stop_words_test)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/abdulraqeeb/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/abdulraqeeb/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('models/20988627_21051268_21049384_NLP.h5')

Metal device set to: Apple M1


In [8]:
import pickle
import os

file_path = os.path.join('data', 'padded_test.pkl')
with open(file_path, 'rb') as file:
    padded_test_run = pickle.load(file)




In [161]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, Dropout, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Setting a random seed for reproducibility
#np.random.seed(42)
#tf.random.set_seed(42)


# Tokenization and Padding
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(processed_test)

vocab_size = len(tokenizer.word_index) + 1
#sequences_train = tokenizer.texts_to_sequences(lemmatized_reviews)
sequences_test = tokenizer.texts_to_sequences(processed_test)


max_sequence_length = 100

#padded_train = pad_sequences(sequences_train, maxlen=max_sequence_length, padding='post')
padded_test = pad_sequences(sequences_test, maxlen=max_sequence_length, padding='post')

In [190]:
import pickle
import os

# Load the padded_test list from the file in the 'data' directory using pickle
padded_test_file_path = os.path.join('data', 'padded_test.pkl')
with open(padded_test_file_path, 'rb') as file:
    padded_test_check = pickle.load(file)

# Load the y_test list from the file in the 'data' directory using pickle
y_test_file_path = os.path.join('data', 'y_test.pkl')
with open(y_test_file_path, 'rb') as file:
    y_test_check = pickle.load(file)





In [9]:
#y_new_np = np.array(y_new)
loss, accuracy = loaded_model.evaluate(padded_test_run, y_test)

# Print the test accuracy
print(f'Test loss: {loss:.4f}, Test accuracy: {accuracy * 100:.2f}%')

2023-07-28 14:11:13.701574: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


782/782 [==============================] - 4s 4ms/step - loss: 0.4627 - accuracy: 0.8661
Test loss: 0.4627, Test accuracy: 86.61%
